# Offer File (AmazonS3)

With this notebook you can create an offer for a file contained in an `AmazonS3` storage in the Dataspace.

## Preperation

Here some values are initialized that will be necessary for the other steps.


### Set up

Import the requests module and assign the base url of the Dataspace as a variable.

In [ ]:
import requests

base_url = "https://vision-x-api.base-x-ecosystem.org"

### Fill in Values

Fill in the values for the variables below.

In [ ]:
# Your JWT recevied from Keycloak via vision-x-auth.base-x-ecosystem.org
token = "ey..."
token_header = {"Authorization": f"Bearer {token}"}
# The name of your Connector
connector_name = "my-connector"
# The URL of your AmazonS3 storage
aws_url = "https://s3.us-east-1.amazonaws.com"
# The region of your AmazonS3 storage
aws_region = "us-east-1"
# The username (Access Key ID) of your AmazonS3 storage account
aws_username = "my-access-key-id"
# The password (Secret Access ID) of your AmazonS3 storage account
aws_password = "my-secret-access-key"
# The bucket where your file is located
aws_bucket = "my-bucket"
# The name of the file
filename = "my-file.txt"

## Create Asset

Here you create an Asset for the file you want to offer. The Asset on one hand describes what kind of data is offered (properties) and on the other hand how the Connector can eventually retrieve the data to transfer it (dataAddress). However, this will be not be available for others as an Offer just yet.

Choose a unique id for your Asset.

In [ ]:
# Some unique ID
asset_id = "my-asset-id"

Creates an Asset with the given values.

In [ ]:
url = f"{base_url}/connectors/{connector_name}/cp/management/v3/assets"
payload = {
  "@context": {},
  "@id": asset_id,
  "properties": {
    "name": f"File {filename}",
    "description": "File offer example with AmazonS3",
    "offerType": "data"
  },
  "dataAddress": {
    "@type": "DataAddress",
    "type": "AmazonS3",
    "objectName": filename,
    "region": aws_region,
    "bucketName": aws_bucket,
    "endpointOverride": aws_url,
    "accessKeyId": aws_username,
    "secretAccessKey": aws_password
  }
}

response = requests.post(url, json=payload, headers=token_header)
response.raise_for_status()
print(f"Created Asset with ID: {asset_id}")

## Create Policy

Here you create a Policy which is basically a collection of terms and conditions. This will later be "assigned" to the Asset in order to make it available for others given they meet the conditions.

Choose a unique id for your Policy.

In [ ]:
# Some unique ID
policy_id = "my-policy-id"

Choose a policy. Here a policy allowing anyone access is chosen.

In [ ]:
# Some ODRL policy
policy = {
  "@type": "odrl:Set",
  "odrl:permission": [
    {
      "odrl:action": "USE"
    }
  ]
}

Creates a Policy with the given values.

In [ ]:
url = f"{base_url}/connectors/{connector_name}/cp/management/v3/policydefinitions"
payload = {
  "@context": {
    "odrl": "http://www.w3.org/ns/odrl/2/"
  },
  "@id": policy_id,
  "policy": policy
}

response = requests.post(url, json=payload, headers=token_header)
response.raise_for_status()
print(f"Created Policy with ID: {policy_id}")

## Create Contract Definition

Here you create a Contract whose role it is to assign Policies to Assets. The contract's "Access Policy" sets the terms and conditions under which the Asset will be visible as an Offer in the Catalog and the "Contract Policy" sets the terms and conditions under which the Negotiation for an Offer will be agreed. The "Assets Selector" determines to which Assets the Policies are "assigned" to.

Choose a unique id for your Policy.

In [ ]:
# Some unique ID
contract_id = "my-contract-id"

Creates a Contract with the given values.

In [ ]:
url = f"{base_url}/connectors/{connector_name}/cp/management/v3/contractdefinitions"
payload = {
  "@context": {
    "odrl": "http://www.w3.org/ns/odrl/2/"
  },
  "@id": contract_id,
  "accessPolicyId": "all",
  "contractPolicyId": policy_id,
  "assetsSelector": {
    "operandLeft": "https://w3id.org/edc/v0.0.1/ns/id",
    "operator": "=",
    "operandRight": asset_id
  }
}

response = requests.post(url, json=payload, headers=token_header)
response.raise_for_status()
print(f"Created Contract Definition with ID: {contract_id}")

If everything was successful the Asset will be now available to others as an Offer in your Catalog.